In [ ]:
#!pip install chromadb sentence-transformers openai

In [1]:
import os
import chromadb
from chromadb import PersistentClient
from sentence_transformers import SentenceTransformer
import openai
import ollama
import requests

In [2]:
# Chroma Config
persist_path = "chroma_db"
collection_name = "class10_english"

In [3]:
# Embedding model
MODEL_NAME = "all-MiniLM-L6-v2"
model = SentenceTransformer(MODEL_NAME)

In [4]:
# LLM Config
USE_OPENAI = False  # Set False to use Ollama
OPENAI_MODEL = "gpt-3.5-turbo"
OLLAMA_MODEL = "llama3.1"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
# Chroma client (🔁 using PersistentClient)
client = PersistentClient(path=persist_path)
collection = client.get_or_create_collection(collection_name)

In [6]:
def retrieve_chunks(query, top_k=3):
    embedding = model.encode([query])[0].tolist()
    results = collection.query(query_embeddings=[embedding], n_results=top_k)
    return results["documents"][0] if results["documents"] else []


In [7]:
def ask_openai(context, query):
    response = openai.ChatCompletion.create(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful study tutor."},
            {"role": "user", "content": f"Context:\n{context}\n\nQuestion:\n{query}"}
        ]
    )
    return response['choices'][0]['message']['content']

In [12]:
def ask_ollama(context, query):
    prompt = f"Context:\n{context}\n\nQuestion: {query}\n\nAnswer:"
    response = requests.post("http://localhost:11434/api/generate", json={
        "model": OLLAMA_MODEL,
        "prompt": prompt,
        "system": "You are a teacher and evaluator of class10_english",
        "stream": False
    })
    return response.json()["response"]

In [13]:
def embed_text(text):
    return model.encode([text])[0].tolist()

In [14]:
query = "Teach me one of the poem from the book."
chunks = retrieve_chunks(query)
context = "\n\n---\n\n".join(chunks) if chunks else ""

# Get answer from LLM
answer = ask_openai(context, query) if USE_OPENAI else ask_ollama(context, query)
print("🧠 Q:", query)
print("💬 A:", answer)

🧠 Q: Teach me one of the poem from the book.
💬 A: Let's take a look at one of the poems in the CBSE Class 10 English textbook. How about we start with "The Lake Isle of Innisfree" by W.B. Yeats?

This poem is actually an excerpt from his collection "New Poems" (1903) and is a beautiful expression of longing for a peaceful life.

Here's the poem:

"The Lake Isle of Innisfree"

I will arise and go now, and go to Innisfree,
And a small cabin build there, of clay and wattles made;
Nine bean-rows will I have there, a hive for the honey-bee,
And live alone in the bee-loud glade.

And I shall have some peace there, for peace comes dropping slow,
Dropping from the veils of the morning to where the cricket sings;
There midnight mass is held, and the bells are ringing
Out to where the lake waters wander from the evening till morning...

Let's break it down line by line and understand its meaning.

How would you like me to explain this poem? Would you like a summary, an analysis of its themes or 